# Import libraries

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
import math

ModuleNotFoundError: No module named 'tensorflow'

# Load Data

In [ ]:

X = pd.read_csv('../dataset/XbaseCher_Bvot.csv', sep=';', index_col=[0,1,2,3])
y = pd.read_csv('../dataset/labels/yDataCher_Canton_DP.csv', sep=';', index_col=[0,1])

yCols = ['target_'+c for c in y.columns]
XCols = X.columns
y.columns = yCols

X.sort_values(['CODCAN', 'CODSUBCOM'])
y.sample(frac=1)

## Select data

In [ ]:
np.random.seed(42)

trainset = pd.DataFrame()
testset = pd.DataFrame()

for canton in X.index.get_level_values('CODCAN'):
    XFiltered = X[X.index.get_level_values('CODCAN')==canton]
    yFiltered = y[y.index.get_level_values('CODCAN')==canton]
    
    XFiltered.sample(frac=1) # shuffle

    nBurTrain = math.ceil(len(XFiltered) * 0.5)
    X_train_canton = XFiltered[:nBurTrain]
    X_test_canton = XFiltered[nBurTrain:]

    colsName = [c+'.'+str(i) for i in range(len(X_train_canton)) for c in X_train_canton.columns.tolist()]
    trainRow = X_train_canton.stack()
    testRow = X_test_canton.stack()
    testRow = testRow.append(pd.Series([0]*(len(trainRow)-len(testRow)), dtype='float64'))
    trainRow.index = colsName
    testRow.index = colsName
    trainRow.name = canton
    testRow.name = canton
    trainset = trainset.append(trainRow)
    testset = testset.append(testRow)

testset = testset[colsName].drop_duplicates().sort_index().fillna(0)
trainset = trainset[colsName].drop_duplicates().sort_index().fillna(0)

In [ ]:
print(trainset.shape, y.shape)
trainset

# Build Model

## Initialization

In [4]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(160,)))
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(20, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

NameError: name 'tf' is not defined

## Training and evaluation

In [ ]:
history = model.fit(trainset, y, batch_size=32, validation_data=(testset, y), epochs=100, verbose=0)

# evaluate the model
_, train_acc = model.evaluate(trainset, y, verbose=0)
_, test_acc = model.evaluate(testset, y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

## Save Model

In [ ]:
model.save('models/Bvot-Canton_Cher')